# Cavity Quantum Electrodynamics Configuration Interaction Singles Theory

In [1]:
"""Tutorial implementing a CQED-CIS program"""

__authors__ = "J. McTague, J. Foley, A. E. DePrince III"
__credits__ = "J. McTague, J. Foley, A. E. DePrince III"
__email__   = "foleyj10@wpunj.edu, deprince@fsu.edu"

__copyright__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "11/04/2021"




# I. Theoretical Overview

The Pauli-Fierz Hamiltonian, $\hat{H}_{PF}$, is defined in the dipole approximation as:
$$\hat{H}_{PF} = \hat{H}_e + \hat{H}_p + \hat{H}_{ep} + \hat{H}_{dse}\tag{1}$$
***
### *Electronic Term*



The term $\hat{H}_e$ represents the electronic Hamiltonian and is given through:
$$\hat{H}_e = \sum^{N_e}_{i} \hat{T}_e(x_i) + \sum^{N_e}_i \sum^{N_N}_A \hat{V}_{eN}(x_i :X_A) + \sum^{N_e}_i \sum^{N_e}_j (\hat{V}_{ee}(x_i, x_j) + V_{N,N}\tag{2}$$
Where $\hat{T}_e(x_i)$ is the electronic kinetic operator for electron $i$, $\hat{V}_{eN}(x_i:X_A)$ is the attractive coulomb operator for electron $i$ and nucleus $A$, $\hat{V}_{ee}(x_i : x_J)$ is the repulsive coulomb operator for electrons $i$ and $j$, and $V_{N,N}$ is the total repulsive coulomb potential between all nuclei. When using the Born-Oppenheimer Approximation, the term $V_{N,N}$ is treated a constant. Additionally, we can neglect the nuclear kinetic energy, and the electron-nuclear attraction will depend upon the coordinates of the fixed nuclear coordinates.
***
### *Photonic Term*

The next term considered within the Pauli-Fierz Hamiltonian is the photonic Hamiltonian, $\hat{H}_p$, which is defined as:
$$\hat{H}_p = \omega\hat{b}^{\dagger{}}\hat{b}\tag{3}$$
Here, $\omega$ represents the frequency of a photon. 

***
### *Bilinear Coupling Term*
The third term in the Hamiltonian is a bilinear coupling term that describes coupling of the photonic and electronic degrees of freedom given by, $\hat{H}_{ep}$ and is defined as:
$$\hat{H}_{ep} = \sqrt{\frac{\omega}{2}}\left(\lambda \cdot (\hat{\mu} - \langle \mu \rangle )\right)\left(\hat{b}^{\dagger{}}+\hat{b}\right)\tag{6}$$

Where $\lambda$ describes the coupling strength, $\hat{\mu}$ is the dipole operator, and $\langle\mu\rangle$ is the molecular dipole expectation value in the ground state. Here, the expectation value has characteristic Cartesian components, denoted by $\xi$. A given dipole operator with $\xi$ components is defined through:
$$\hat{\mu}^{\xi} = \sum^{N_e}_i \hat{\mu}^\xi (x_i)+{\mu}_{nuc}^\xi \mid \xi \in \{x,y,z\}\tag{7}$$

Where $\mu^{\xi}(x_i)$ is defined as an operator that depends upon electronic coordinates, and $\mu_{nuc}^{\xi}$ is the nuclear dipole moment where the Cartesian components are treated as functions of the nuclear coordinate.

***
### *Dipole Self Energy Term*
The final component of the Hamiltonian, $\hat{H}_{dse}$ is the dipole self energy term, and is defined through:
$$\hat{H}_{dse} = \frac{1}{2}\left(\lambda \cdot (\hat{\mu} - \langle\mu\rangle)\right)^2\tag{8}$$

Here, $\lambda$ represents the polarization term, $\hat{\mu}$ represents the molecular dipole operator and $\langle\mu\rangle$ represents the ground state molecular expeectation value.
***

***
### *CQED-CIS Ansatz*
The polaritonic energy eigenfunctions for state $I$ in the
NH-CQED-CIS ansatz can be written as a linear combination of the CQED-RHF reference and products of all possible single electron excitations out of the CQED-RHF reference.  The CQED-RHF reference involves the product of an electronic Slater determinant with the photon vacuum state $|\Phi_o\rangle |0\rangle$, so single excitations can occur as electronic excitations from an occupied orbital $\phi_i$ to a virtual orbital $\phi_a$, the raising of the photon number state from $|0\rangle \rightarrow |1\rangle$, or both.  We therefore write the NH-QED-CIS wavefunction for state $I$ as
$$\Psi_I = c_0^0 |\Phi_0\rangle |0\rangle + 
c_0^1 |\Phi_0\rangle |1\rangle +
\sum_{i,a} c_{ia}^0 |\Phi_i^a\rangle |0\rangle +
\sum_{i,a} c_{ia}^1 |\Phi_i^a\rangle |1\rangle.\tag{9}$$
where the coefficients $c$ denote the contribution of a given term to the wavefunction and we have denoted the electronic excitations in the subscript and the photonic excitations in the superscript of these coefficients.  \textcolor{black}{For the case of multiple modes, the photonic basis states will be augmented to consider all possible combinations of the occupations of those modes within a maximum photon number}.  These coefficients, and the corresponding energy eigenvalues for a given NH-CQED-CIS state $I$, may be obtained by diagonalizing the Hamiltonian matrix built in the basis shown in Eq. (XX).  We spin adapt this basis such that $|\Phi_i^a\rangle = \frac{1}{\sqrt{2}}\left(|\Phi_{i\alpha}^{a \alpha} \rangle + |\Phi_{i\beta}^{a\beta}\rangle \right)$, where $\alpha$ and $\beta$ label the spin orbitals as being occupied by spin-up and spin-down electrons, respectively.  There are three classes of matrix elements that contribute to the Hamiltonian matrix, and we we write each class of matrix elements after shifting the total Hamiltonian in Eq. (1) by the energy of the CQED-RHF reference $E_{CQED-RHF}$. 


The matrix elements involving the CQED-RHF electronic Slater determinant $|\Phi_0\rangle$ and photonic states $|s\rangle$ and $|t\rangle$, where $s,t \in \{0,1\}$ involve only the photonic energy,

$$\langle s | \langle \Phi_0 | \hat{H}-E_{CQED-RHF} | \Phi_0 \rangle | t \rangle = 
    \omega \: t  \delta_{st}.\tag{10}$$


Matrix elements coupling $|\Phi_0\rangle|s\rangle$ to $|\Phi_i^a\rangle |t\rangle$ involve only the $\hat{H}_{ep}$ contributions:

\begin{align}
  &  \langle s | \langle \Phi_0 | \hat{H}-E_{CQED-RHF} | \Phi_i^a \rangle | t \rangle = \nonumber \\
   & -\sqrt{\omega} \sqrt{t+1} \delta_{s,t+1} \sum_{\xi} \lambda^{\xi} \mu_{ia}^{\xi} \\
   &- -\sqrt{\omega} \sqrt{t} \delta_{s,t-1} \sum_{\xi} \lambda^{\xi} \mu_{ia}^{\xi}. \tag{11}
\end{align}

Matrix elements coupling different singly excited electronic and/or photonic states involve all terms of the Hamiltonian, including the canonical CIS terms:
\begin{align}
    &\langle s \langle \Phi_i^a | \hat{H}-E_{CQED-RHF} | \Phi_j^b \rangle | t \rangle \nonumber \\
    & =   \left(\epsilon_a - \epsilon_i + d_c + \omega \: t \right)\delta_{ij} \delta_{ab} \delta_{st} \nonumber \\
    &+ \delta_{st} \left( 2(ia|jb) - (ij|ab) \right) \nonumber \\
    &+ 2\delta_{st} \sum_{\xi,\xi'} \lambda^{\xi} \lambda^{\xi'} \mu^{\xi}_{ia} \mu^{\xi}_{jb} \nonumber \\
    &- \delta_{st} \sum_{\xi,\xi'} \lambda^{\xi} \lambda^{\xi'} \mu^{\xi}_{ij} \mu^{\xi}_{ab} \nonumber \\
    &+ \sqrt{t+1} \: \delta_{s,t+1} \delta_{ij} \delta_{ab} \sqrt{\frac{\tilde{\omega}}{2}} \lambda \cdot \langle \mu \rangle \nonumber \\
    &+ \sqrt{t} \: \delta_{s,t-1} \delta_{ij} \delta_{ab} \sqrt{\frac{ \omega}{2}} \lambda \cdot \langle \mu \rangle \nonumber \\
    &- \sqrt{t+1} \: \delta_{s,t+1} \delta_{ij} \delta_{ab} \sqrt{\frac{ \omega}{2}} \sum_{\xi} \sum_{k} \lambda^{\xi} \mu^{\xi}_{kk}  \\
    &- \sqrt{t} \: \delta_{s,t-1} \delta_{ij} \delta_{ab} \sqrt{\frac{ \omega}{2}} \sum_{\xi} \sum_{k} \lambda^{\xi} \mu^{\xi}_{kk} \nonumber \\
    &- \sqrt{t+1} \: \delta_{s,t+1} \delta_{ij}  \sqrt{\frac{ \omega}{2}} \sum_{\xi} \lambda^{\xi} \mu^{\xi}_{ab}  \nonumber\\
    &- \sqrt{t} \: \delta_{s,t-1} \delta_{ij} \sqrt{\frac{ \omega}{2}} \sum_{\xi} \lambda^{\xi} \mu^{\xi}_{ab}   \nonumber\\
    &+ \sqrt{t+1} \: \delta_{s,t+1} \delta_{ab} \sqrt{\frac{ \omega}{2}} \sum_{\xi} \lambda^{\xi} \mu^{\xi}_{ij}  \nonumber \\
    &+ \sqrt{t} \: \delta_{s,t-1} \delta_{ab} \sqrt{\frac{ \omega}{2}} \sum_{\xi} \lambda^{\xi} \mu^{\xi}_{ij}.  \nonumber
\end{align}

## II. Implementation

Using the above overview, we will implement the CQED-CIS method using <span style="font-variant: small-caps"> Psi4 </span> and NumPy.  First, we need to import these Python modules: 

In [2]:
# ==> Import Psi4 & NumPy <==
import psi4
import numpy as np
import scipy.linalg as la
import time
from helper_cqed_rhf import cqed_rhf


Next, using what you learned in the previous tutorial module, set the following <span style="font-variant: small-caps"> Psi4 </span> and molecule options.

Memory & Output specifications:
- Give 500 Mb of memory to Psi4
- Set Psi4 output file to "output.dat"
- Set a variable `numpy_memory` to an acceptable amount of available memory for the working computer to use for storing tensors

Molecule definition:
*(Do we want to do formaldehyde instead?)*

- Define the "physicist's water molecule" (O-H bond length = 1.1 Angstroms, HOH bond angle = 104 degrees)
- Molecular symmetry C1

Computation options:
- basis set cc-pVDZ
- SCF type PK
- Energy convergence criterion to 0.00000001


In [18]:
# ==> Set Basic Psi4 Options <==
# Memory specification
psi4.set_memory(int(5e8))
numpy_memory = 2

# Set output file
psi4.core.set_output_file('output.dat', False)

# Define Physicist's water -- don't forget C1 symmetry!
#molecule_string = """
#O
#H 1 1.1
#H 1 1.1 2 104
#symmetry c1
#"""

#options_dict = {
#    'basis': 'cc-pvdz',
#    'scf_type': 'pk',
#    'e_convergence': 1e-8
#}


molecule_string = """
Mg
H 1 2.2
symmetry c1
1 1
"""

# options dict
options_dict = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}



psi4.set_options(options_dict)

mol = psi4.geometry(molecule_string)
scf_e, wfn = psi4.energy('scf', return_wfn=True)
print(scf_e)

# electric field for H2O - polarized along z-axis with mangitude 0.05 atomic units
lam_h2o = np.array([0.0, 0.0, 0.0125])
lambda_vector = np.copy(lam_h2o)
# run cqed_rhf method
cqed_rhf_dict = cqed_rhf(lam_h2o, molecule_string, options_dict)

omega_val = 4.75 / psi4.constants.Hartree_energy_in_eV



-199.86395910419174

Start SCF iterations:

Canonical RHF One-electron energy = -281.6658725094007423
CQED-RHF One-electron energy      = -281.6660217366671191
Nuclear repulsion energy          = 2.8864211491090908
Dipole energy                     = 0.0009465637014397
SCF Iteration   1: Energy = -199.8633303817348406   dE = -1.99863E+02   dRMS = 2.52850E-05
SCF Iteration   2: Energy = -199.8633315502318339   dE = -1.16850E-06   dRMS = 9.55045E-06
SCF Iteration   3: Energy = -199.8633316895821395   dE = -1.39350E-07   dRMS = 3.61517E-06
SCF Iteration   4: Energy = -199.8633317095171549   dE = -1.99350E-08   dRMS = 1.37921E-06
SCF Iteration   5: Energy = -199.8633317124265432   dE = -2.90939E-09   dRMS = 5.28065E-07
SCF Iteration   6: Energy = -199.8633317128551425   dE = -4.28599E-10   dRMS = 2.02751E-07
SCF Iteration   7: Energy = -199.8633317129182956   dE = -6.31530E-11   dRMS = 7.79875E-08
SCF Iteration   8: Energy = -199.8633317129277884   dE = -9.49285E-12   dRMS = 3.00308E-08
SC

We need to prepare the arrays that will be used in the elements of the CQED-CIS matrix.  We will need access to 2-electron repulsion integrals, dipole integrals, and quadrupole integrals all in the CQED-RHF basis, and we will also need the orbital energies that arise from CQED-RHF.  The following block will run a CQED-RHF calculation on the molecule defined above to obtain the CQED-RHF orbitals and orbital energies needed to transform all integrals to the CQED-RHF basis.

In [19]:
cqed_rhf_dict = cqed_rhf(lam_h2o, molecule_string, options_dict)

# grab necessary quantities from cqed_rhf_dict
scf_e = cqed_rhf_dict["RHF ENERGY"]
cqed_scf_e = cqed_rhf_dict["CQED-RHF ENERGY"]
wfn = cqed_rhf_dict["PSI4 WFN"]
C = cqed_rhf_dict["CQED-RHF C"]
eps = cqed_rhf_dict["CQED-RHF EPS"]
cqed_rhf_dipole_moment = cqed_rhf_dict["CQED-RHF DIPOLE MOMENT"]

# Create instance of MintsHelper class
mints = psi4.core.MintsHelper(wfn.basisset())

# Grab data from wavfunction

# number of doubly occupied orbitals
ndocc = wfn.nalpha()

# total number of orbitals
nmo = wfn.nmo()

# number of virtual orbitals
nvirt = nmo - ndocc

# need to update the Co and Cv core matrix objects so we can
# utlize psi4s fast integral transformation!

# collect rhf wfn object as dictionary
wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# update wfn_dict with orbitals from CQED-RHF
wfn_dict["matrix"]["Ca"] = C
wfn_dict["matrix"]["Cb"] = C
# update wfn object
wfn = psi4.core.Wavefunction.from_file(wfn_dict)

# occupied orbitals as psi4 objects but they correspond to CQED-RHF orbitals
Co = wfn.Ca_subset("AO", "OCC")

# virtual orbitals same way
Cv = wfn.Ca_subset("AO", "VIR")


Now that we have the correct CQED-RHF transformation vectors (`Co` for the occupied subspace and `Cv` for the virtual subspace), we will obtain the 2-electron repulsion integrals (2ERI) in the CQED-RHF basis.  We can make use of helper functions associated with the Psi4 mints library to transform the integrals from the AO to CQED-RHF basis given the transformation vectors as arguments.  We need 2ERIs of the type $(ia|jb)$ and $(ij|ab)$ for the CQED-CIS equations, and we will store these integrals in arrays called `ovov` and `oovv`, respectively.

In [ ]:
# 2 electron integrals in CQED-RHF basis

# use built-in mints function to build the (ov|ov)
# integrals in the CQED-RHF basis using Co and Cv
ovov = np.asarray(mints.mo_eri(Co, Cv, Co, Cv))

# use built-in mints function to build the (oo|vv)
# integrals in the CQED-RHF basis using Co and Cv
oovv = np.asarray(mints.mo_eri(Co, Co, Cv, Cv))



We need the occupied and virtual CQED-RHF orbital energies, $\epsilon_i$ and $\epsilon_a$ respectively, which we will store in arrays called `\eps_o` and `eps_v`.

In [ ]:
# strip out occupied orbital energies, eps_o spans 0..ndocc-1
eps_o = eps[:ndocc]

# strip out virtual orbital energies, eps_v spans 0..nvirt-1
eps_v = eps[ndocc:]


Terms of the form $\lambda^{\xi} \mu_{pq}^{\xi}$ require dipole integrals in the CQED-RHF basis dotted into the electric field vector $\vec{\lambda}$. 

Here we will obtain the Cartesian components of the dipole integrals in the AO basis and store them in arrays `mu_ao_x`, `mu_ao_y`, and `mu_ao_z`.  We will then transform them to the CQED-RHF basis using the transformation vectors stored in an array called `C`. Finally, we will accumulate the dot product of these integrals and the $\lambda$ vector and store the result in an array called `l_dot_mu_el`.

In [ ]:
# dipole arrays in AO basis
mu_ao_x = np.asarray(mints.ao_dipole()[0])
mu_ao_y = np.asarray(mints.ao_dipole()[1])
mu_ao_z = np.asarray(mints.ao_dipole()[2])

# transform dipole array to CQED-RHF basis
mu_cmo_x = np.dot(C.T, mu_ao_x).dot(C)
mu_cmo_y = np.dot(C.T, mu_ao_y).dot(C)
mu_cmo_z = np.dot(C.T, mu_ao_z).dot(C)

# \lambda \cdot \mu_{el}
# e.g. line 4 Eq. (18) in [McTague:2021:ChemRxiv]
l_dot_mu_el = lambda_vector[0] * mu_cmo_x
l_dot_mu_el += lambda_vector[1] * mu_cmo_y
l_dot_mu_el += lambda_vector[2] * mu_cmo_z

Additionally, we will need the electric field vector dotted into the nuclear dipole moment and the dipole expectation value computed at the CQED-RHF level.

In [ ]:

# Extra terms for Pauli-Fierz Hamiltonian
# nuclear dipole
mu_nuc_x = mol.nuclear_dipole()[0]
mu_nuc_y = mol.nuclear_dipole()[1]
mu_nuc_z = mol.nuclear_dipole()[2]

# l \cdot \mu_nuc for d_c
l_dot_mu_nuc = lambda_vector[0] * mu_nuc_x
l_dot_mu_nuc += lambda_vector[1] * mu_nuc_y
l_dot_mu_nuc += lambda_vector[2] * mu_nuc_z


# \lambda \cdot < \mu >
# e.g. line 6 of Eq. (18) in [McTague:2021:ChemRxiv]
l_dot_mu_exp = 0.0
for i in range(0, 3):
    l_dot_mu_exp += lambda_vector[i] * cqed_rhf_dipole_moment[i]


# dipole constants to add to E_CQED_CIS,
#  0.5 * (\lambda \cdot \mu_{nuc})** 2
#      - (\lambda \cdot <\mu> ) ( \lambda \cdot \mu_{nuc})
# +0.5 * (\lambda \cdot <\mu>) ** 2
# Eq. (14) of [McTague:2021:ChemRxiv]
d_c = (
    0.5 * l_dot_mu_nuc ** 2 - l_dot_mu_nuc * l_dot_mu_exp + 0.5 * l_dot_mu_exp ** 2
)

# check to see if d_c what we have from CQED-RHF calculation
assert np.isclose(d_c, cqed_rhf_dict["DIPOLE ENERGY"])




In [ ]:
# create Hamiltonian for elements H[ias, jbt]
Htot = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
Hep = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
H1e = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
H2e = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
H2edp = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
Hp = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)



In [ ]:
# elements corresponding to <s|<\Phi_0 | H | \Phi_0>|t>
# Eq. (16) of [McTague:2021:ChemRxiv]
Hp[0, 0] = 0.0
Hp[1, 1] = omega_val


In [ ]:

# elements corresponding to <s|<\Phi_0 | H | \Phi_i^a>|t>
# Eq. (17) of [McTague:2021:ChemRxiv]
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        Hep[0, ia1] = Hep[ia1, 0] = (
            -np.sqrt(omega_val) * l_dot_mu_el[i, A]
        )
        Hep[1, ia0] = Hep[ia0, 1] = (
            -np.sqrt(omega_val) * l_dot_mu_el[i, A]
        ) 



In [ ]:
# elements corresponding to <s|<\Phi_i^a| H | \Phi_j^b|t>
# Eq. (18) of [McTague:2021:ChemRxiv]
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        for s in range(0, 2):
            ias = 2 * (i * nvirt + a) + s + 2

            for j in range(0, ndocc):
                for b in range(0, nvirt):
                    B = b + ndocc
                    for t in range(0, 2):
                        jbt = 2 * (j * nvirt + b) + t + 2
                        # ERIs
                        H2e[ias, jbt] = (
                            2.0 * ovov[i, a, j, b] - oovv[i, j, a, b]
                        ) * (s == t)
                        # 2-electron dipole terms
                        # ordinary
                        H2edp[ias, jbt] += (
                            2.0 * l_dot_mu_el[i, A] * l_dot_mu_el[j, B]
                        ) * (s == t)
                        # exchange
                        H2edp[ias, jbt] -= (
                            l_dot_mu_el[i, j] * l_dot_mu_el[A, B] * (s == t)
                        )
                        # orbital energies from CQED-RHF
                        H1e[ias, jbt] += eps_v[a] * (s == t) * (a == b) * (i == j)
                        H1e[ias, jbt] -= eps_o[i] * (s == t) * (a == b) * (i == j)
                        # photonic and dipole energy term
                        Hp[ias, jbt] += (
                            (omega_val * t) * (s == t) * (i == j) * (a == b)
                        )
                        # bilinear coupling - off-diagonals first
                        Hep[ias, jbt] += (
                            np.sqrt(t + 1)
                            * np.sqrt(omega_val / 2)
                            * l_dot_mu_el[i, j]
                            * (s == t + 1)
                            * (a == b)
                        )
                        Hep[ias, jbt] += (
                            np.sqrt(t)
                            * np.sqrt(omega_val / 2)
                            * l_dot_mu_el[i, j]
                            * (s == t - 1)
                            * (a == b)
                        )
                        Hep[ias, jbt] -= (
                            np.sqrt(t + 1)
                            * np.sqrt(omega_val / 2)
                            * l_dot_mu_el[A, B]
                            * (s == t + 1)
                            * (i == j)
                        )
                        Hep[ias, jbt] -= (
                            np.sqrt(t)
                            * np.sqrt(omega_val / 2)
                            * l_dot_mu_el[A, B]
                            * (s == t - 1)
                            * (i == j)
                        )
                        # now handle diagonal in electronic term
                        if a == b and i == j and s == t + 1:
                            # l dot <mu> term
                            Hep[ias, jbt] += (
                                np.sqrt(t + 1)
                                * np.sqrt(omega_val / 2)
                                * l_dot_mu_exp
                            )
                            # l dot mu terms
                            for k in range(0, ndocc):
                                # sum over occupied indices
                                Hep[ias, jbt] -= (
                                    np.sqrt(t + 1)
                                    * np.sqrt(omega_val / 2)
                                    * l_dot_mu_el[k, k]
                                )

                        # now handle diagonal in electronic term
                        if a == b and i == j and s == t - 1:
                            # l dot <mu> term
                            Hep[ias, jbt] += (
                                np.sqrt(t) * np.sqrt(omega_val / 2) * l_dot_mu_exp
                            )
                            # l dot mu terms
                            for k in range(0, ndocc):
                                # sum over occupied indices
                                Hep[ias, jbt] -= (
                                    np.sqrt(t)
                                    * np.sqrt(omega_val / 2)
                                    * l_dot_mu_el[k, k]
                                )





In [ ]:
# Form Htot from sum of all terms
Htot = Hp + Hep + H1e + H2e + H2edp


ECIS, L_CCIS = np.linalg.eigh(Htot)
R_CCIS = np.copy(L_CCIS)

cqed_cis_dict = {
    'RHF ENERGY' : scf_e,
    'CQED-RHF ENERGY' : cqed_scf_e,
    'CQED-CIS ENERGY' : ECIS,
    'CQED-CIS R VECTORS' : R_CCIS,
    'CQED-CIS L VECTORS' : L_CCIS,
    'CQED-CIS H MATRIX' : Htot
}



### References
1. Pauli-Fierz Hamiltonian and CQED-RHF Equations
    - [[Haugland:2020:041043](https://journals.aps.org/prx/pdf/10.1103/PhysRevX.10.041043)] T. S. Haughland, E. Ronco, E. F. Kjonstad, A. Rubio, H. Koch, *Phys. Rev. X*, **10**, 041043 (2020)
    - [[DePrince:2021:094112]](https://aip.scitation.org/doi/10.1063/5.0038748) A. E. DePrince III, *J. Chem. Phys.* **154**, 094113 (2021).
2. Detailed CQED-RHF and CQED-CIS equations and overview of algorithm
    - [[McTague:2021:ChemRxiv](https://chemrxiv.org/engage/chemrxiv/article-details/611fa8d08a6faa13229c8be6)] J. McTague, J. J. Foley IV, *ChemRxiv*,
doi: 10.33774/chemrxiv-2021-0gpz8 (2021)

In [20]:
print(cqed_cis_dict['CQED-CIS ENERGY'][1])

0.16563135967439932
